# Jupyter + Spark on XSEDE

Here's a little notebook that demonstrates how to use Spark from a Python Jupyter notebook on the [XSEDE](https://xsede.org) compute platform. The imports are really all you need to know to allow Jupyter to see pyspark, but what follows is a short example of using Spark.

## Setup

First ssh to the Pittsburgh Supercomputer Center XSEDE and get this notebook:

    ssh username@bridges.psc.xsede.org
    git clone https://github.com/edsu/jupyter-spark-xsede
    
Then head over to XSEDE's [On Demand Interface](https://ondemand.bridges.psc.edu/) and start a Juypyter session, under *Interactive Session* in the menu bar. Then navigate to where you cloned this notebook.

## Start Spark

Before we can interact with Spark you need to adjust the path a bit so Jupyter can find pyspark:

In [1]:
import sys

sys.path.append("/opt/packages/spark/latest/python/lib/py4j-0.10.4-src.zip")
sys.path.append("/opt/packages/spark/latest/python/")
sys.path.append("/opt/packages/spark/latest/python/pyspark")

Now you need to create the Spark context:

In [2]:
from pyspark import SparkContext

sc = SparkContext()

## Word Counts in Shakespeare

Load all of Skakespeare's works that are conveniently stored in a single text file:

In [3]:
text = sc.textFile("Complete_Shakespeare.txt")

Get all the words:

In [4]:
words = text.flatMap(lambda line: line.lower().split())
words.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'the',
 'complete',
 'works',
 'of',
 'william']

In [5]:
print("{} unique words".format(words.distinct().count()))

59722 unique words


Generate the word counts:

In [6]:
word_counts = words.map(lambda w: (w, 1)).reduceByKey(lambda a, b: a + b)
word_counts.take(10)

[('the', 27730),
 ('project', 320),
 ('gutenberg', 250),
 ('ebook', 13),
 ('of', 18126),
 ('complete', 243),
 ('works', 268),
 ('william', 311),
 ('shakespeare,', 2),
 ('by', 4310)]

Sort the word counts in descending order:

In [7]:
word_counts = word_counts.sortBy(lambda x: x[1], ascending=False)

Print the top 5

In [8]:
print(word_counts.take(5))

[('the', 27730), ('and', 26099), ('i', 19540), ('to', 18762), ('of', 18126)]


## Stopwords

Let's try again by removing stopwords:

In [9]:
stopwords = open('stopwords_en.txt').read().split()
stopwords[0:5]

['a', "a's", 'able', 'about', 'above']

In [10]:
words2 = words.filter(lambda w: w not in stopwords)
print("{} unique words".format(words2.distinct().count()))

59265 unique words


In [11]:
word_counts2 = words2.map(lambda w: (w, 1)).reduceByKey(lambda a, b: a + b)
word_counts2 = word_counts2.sortBy(lambda x: x[1], ascending=False)
print(word_counts2.take(20))

[('thou', 5138), ('thy', 4028), ('good', 2560), ('enter', 2013), ('hath', 1902), ('thee', 1794), ('king', 1698), ('make', 1599), ('you,', 1479), ("'tis", 1367), ('give', 1288), ('love', 1279), ('sir,', 1235), ('me,', 1219), ("th'", 1146), ('man', 1033), ('o,', 1008), ('lord,', 977), ('time', 936), ('doth', 912)]
